<a href="https://colab.research.google.com/github/TBLinhDan/RAG_on_PDF_Q-A_using_LLM/blob/feature%2Fmodule1-week4-RAG_on_PDF_Q%26A_using_LLM/RAG_on_PDF_Q%26A_using_LLM_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Tích hợp chương trình RAG** (sử dụng thư viện ***LangChain*** có thể trả về output đúng như dự định) **với một giao diện Chat** (sử dụng thư viện ***Chainlit*** chuyên cho ChatBot) **để có được một Ứng dụng Chat hoàn chỉnh.**

**1. Tải các gói thư viện:**

In [ ]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

**2. Import các gói thư viện cần thiết:**

In [ ]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

**3. Khởi tạo bộ tách văn bản (text splitter), tách file.pdf ra thành các đoạn văn bản nhỏ, mỗi đoạn văn bản nhỏ được coi như là một tài liệu trong cơ sở dữ liệu.**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

**Khởi tạo instance vectorization, chuyển đổi các văn bản thành các vector, giúp dễ dàng và chính xác trong thục hiện truy vấn.**

In [ ]:
embedding = HuggingFaceEmbeddings() # chuyển đổi văn bản thành vector

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**4. Xây dựng hàm xử lý file.pdf input đầu vào (đọc và tách văn bản):**

In [ ]:
def process_file(): #file: AskFileResponse
	if file.type == "application/pdf":
		Loader = PyPDFLoader
	#elif file.type == "text/plain":
		#Loader = TextLoader

	loader = Loader(file.path)
	documents = loader.load()
	docs = text_splitter.split_documents(documents)
	for i, doc in enumerate(docs):
		doc.metadata["source"] = f"source_{i}"
	return docs


**5. Xây dựng hàm khởi tạo Chroma database (khởi tạo vector database):**

In [ ]:
def get_vector_db(file): # AskFileResponse
    docs = process_file(file)
    cl.user_session.set("docs", docs)
    vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
    return vector_db

**6. Khởi tạo mô hình ngôn ngữ lớn:**

In [ ]:
def get_huggingface_llm(model_name: str = "lmsys/vicuna-7b-v1.5",
			            max_new_token: int = 512):
    nf4_config = BitsAndBytesConfig(
	                load_in_4bit=True,
	                bnb_4bit_quant_type="nf4",
	                bnb_4bit_use_double_quant=True,
	                bnb_4bit_compute_dtype=torch.bfloat16
	                )
    model = AutoModelForCausalLM.from_pretrained(
	                model_name,
	                quantization_config=nf4_config,
	                low_cpu_mem_usage=True
	                )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_pipeline = pipeline(
	                "text-generation",
	                model=model,
	                tokenizer=tokenizer,
	                max_new_tokens=max_new_token,
	                pad_token_id=tokenizer.eos_token_id,
	                device_map="auto"
	                )

    llm = HuggingFacePipeline(pipeline = model_pipeline,)
    return llm

LLM = get_huggingface_llm()

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

**7. Khởi tạo welcome message:**

In [ ]:
welcome_message = """ Welcome to the PDF QA! To get started:
1. Upload a PDF or text file
2. Ask a question about the file
"""

**8. Khởi tạo hàm on_chat_start:**

In [ ]:
!pip install chainlit
import chainlit as cl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for literalai: filename=literalai-0.0.607-py3-none-any.whl size=64143 sha256=79a9caf42135aa08ae91f530ae5a09f61e55658ff281599a131eaf21ebfd7df7
  Stored in directory: /root/.cache/pip

In [ ]:
@cl.on_chat_start
async def on_chat_start():
    files = None
    while files is None:
	    files = await cl.AskFileMessage(
	    content=welcome_message,
    	accept=["text/plain","application/pdf"],
	    max_size_mb=20,
	    timeout=180,
        ).send()
    file = files[0]

    msg = cl.Message(content=f"Processing'{ file.name}'...", disable_feedback=True)
    await msg.send()

    vector_db = await cl.make_async(get_vector_db)(file)

    message_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
                memory_key="chat_history",
     	        output_key="answer",
     	        chat_memory=message_history,
     	        return_messages=True,
                )
    retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={'k': 3})

    chain = ConversationalRetrievalChain.from_llm(
	            llm =LLM,
	            chain_type="stuff",
	            retriever=retriever,
	            memory=memory,
	            return_source_documents=True
                )

    msg.content = f"'{ file.name}' processed.You can now ask questions!"
    await msg.update()

    cl.user_session.set("chain", chain)

**9. Khởi tạo hàm on_message:**

In [ ]:
@cl.on_message
async def on_message(message: cl.Message):
    chain = cl.user_session.get("chain")
    cb = cl.AsyncLangchainCallbackHandler()
    res = await chain.ainvoke(message.content, callbacks=[cb])
    answer = res["answer"]
    source_documents = res["source_documents"]
    text_elements = []

    if source_documents:
	    for source_idx, source_doc in enumerate(source_documents):
	        source_name = f"source_{source_idx}"
	        text_elements.append(
		    cl.Text(content=source_doc.page_content,
			name=source_name)
	        )
	    source_names = [text_el.name for text_el in text_elements]

	    if source_names:
	        answer += f"\ nSources: {', '.join(source_names)}"
	    else :
	        answer += "\nNo sources found"

    await cl.Message(content=answer, elements=text_elements).send()



**10. Chạy chainlit app:** (Thực hiện trên máy Local)

In [ ]:
!chainlit run app.py --host 0.0.0.0 --port 8000 &>/content/logs.txt &

**11. Expose localhost thành public host bằng localtunnel:**


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen ('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

!lt --port 8000 -- subdomain aivn-simple-rag

Password/Enpoint IP for localtunnel is: 34.74.126.141
/bin/bash: line 1: lt: command not found


**Khi chạy xong các bước trên, truy cập vào đường dẫn tạm localtunnel, nhập password được cung cấp để truy cập vào giao diện web.**